In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import jieba
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from ark_nlp.model.ner.global_pointer_bert import GlobalPointerBert
from ark_nlp.model.ner.global_pointer_bert import GlobalPointerBertConfig
from ark_nlp.model.ner.global_pointer_bert import Dataset
from ark_nlp.model.ner.global_pointer_bert import Task
from ark_nlp.model.ner.global_pointer_bert import get_default_model_optimizer
from ark_nlp.model.ner.global_pointer_bert import Tokenizer

In [ ]:
# 目录地址

train_data_path = '../data/source_datasets/CMeEE/CMeEE_train.json'
dev_data_path = '../data/source_datasets/CMeEE/CMeEE_dev.json'

### 一、数据读入与处理

#### 1. 数据读入

In [ ]:
train_data_df = pd.read_json(train_data_path)
dev_data_df = pd.read_json(dev_data_path)

In [ ]:
train_data_df = train_data_df.rename(columns={'entities': 'label'})
dev_data_df = dev_data_df.rename(columns={'entities': 'label'})

In [ ]:
train_data_df = train_data_df.loc[:,['text', 'label']]
train_data_df['label'] = train_data_df['label'].apply(lambda x: str(x))

dev_data_df = dev_data_df.loc[:,['text', 'label']]
dev_data_df['label'] = dev_data_df['label'].apply(lambda x: str(x))

In [ ]:
ner_train_dataset = Dataset(train_data_df)
ner_dev_dataset = Dataset(dev_data_df)

#### 2. 词典创建和生成分词器

In [ ]:
# 可以先创建词典，再加载入分词器
# 也可以使用分词器自动加载
# bert_vocab = transformers.AutoTokenizer.from_pretrained('nghuyong/ernie-1.0')
# tokenizer = TransfomerTokenizer(bert_vocab, max_seq_len=30)

In [ ]:
tokenizer = Tokenizer(vocab='nghuyong/ernie-1.0', max_seq_len=100)

#### 3. ID化

In [ ]:
ner_train_dataset.convert_to_ids(tokenizer)
ner_dev_dataset.convert_to_ids(tokenizer)

<br>

### 二、模型构建

#### 1. 模型参数设置

In [ ]:
config = GlobalPointerBertConfig.from_pretrained('nghuyong/ernie-1.0', 
                                                 num_labels=len(ner_train_dataset.cat2id))

#### 2. 模型创建

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dl_module = GlobalPointerBert.from_pretrained('nghuyong/ernie-1.0', 
                                              config=config)

<br>

### 三、任务构建

#### 1. 任务参数和必要部件设定

In [ ]:
# 设置运行次数
num_epoches = 1
batch_size = 32

In [ ]:
optimizer = get_default_model_optimizer(dl_module)

#### 2. 任务创建

In [ ]:
model = Task(dl_module, optimizer, 'gpce', cuda_device=0)

#### 3. 训练

In [ ]:
model.fit(ner_train_dataset, 
          ner_dev_dataset,
          lr=1e-5,
          epochs=num_epoches, 
          batch_size=batch_size
         )

<br>

### 四、模型验证与保存

#### 1. 模型验证

In [ ]:
from ark_nlp.model.ner.global_pointer_bert import Predictor

In [ ]:
ner_predictor_instance = Predictor(model.module, tokenizer, ner_train_dataset.cat2id)

In [ ]:
ner_predictor_instance.predict_one_sample('今天扎的维生素K1')

#### 2. Batch模型验证

In [ ]:
test_data_df = pd.read_json('../data/source_datasets/CMeEE/CMeEE_test.json')
test_data_df = test_data_df.loc[:,['text']]

ner_test_dataset = Dataset(test_data_df, categories=ner_train_dataset.categories, is_test=True)

In [ ]:
# predict_label = ner_predictor_instance.predict_batch(ner_test_dataset)

#### 3. 多样本验证

In [ ]:
test_data_df = pd.read_json('../data/source_datasets/CMeEE/CMeEE_test.json')
test_data_df = test_data_df.loc[:,['text']]

record_ = []
for text_ in test_data_df['text'].to_list():
    record_.append([text_, ner_predictor_instance.predict_one_sample(text_)])

<br>

### 五、模型测试报告

In [ ]:
1. 基本功能测试 通过
2. one sample predict 通过
3. batch predict  无batch接口
4. 多样本验证  通过